## Problem formulation

$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\P}{\mathbb{P}}$
### Problem
For $i \in \{0, \ldots N -1\}$ consider independent 
$$Y_i \sim Norm(0,1) \tag{i}\label{i}$$
$$ Z^{(0)}_i,Z^{(1)}_i \sim Norm(Y_i, \sigma^2) \tag{ii}$$
We measure all $Z$, we want to estimate $\sigma^2$ and we don't care about the values of $Y$.

### Frequentist simple maximal likelihood estimator
In the simplistic frequentist approach we do not even need ($\ref{i}$). We consider $M_i, \sigma^2$ as unknown parameters and look for them by maximalizing the likelihood. We get 
$$\hat\sigma_{mle}^2 = \frac{1}{4} \sum_i \frac{(X_i - Y_i)^2}{N} \to \frac 1 2 \sigma^2.$$
so it is not very satisfactory.

### Bayesian approach
For a Bayesian approach we must furthermore specify a prior on $\sigma^2$. It is easier for us to work with $\log(\sigma^2)\in \R$ rather than $\sigma^2 \in \R_{+}$ and we choose inproper
$$\P\big[\log(\sigma^2) \in dx\big] \propto dx \tag{iii}$$

**Remark:** The prior does not save us from the above problem. If we maximize the joint posterior on $\sigma^2, M$, we get basically the same result. 

## Imports and setup

In [1]:
import numpy as np

import matplotlib.pyplot as pl
%matplotlib inline

In [2]:
import tensorflow as tf
import tensorflow.distributions as tfd
from tensorflow.distributions import Normal
from tensorflow.nn import softmax
from tensorflow import exp, log, ones, zeros

tf.enable_eager_execution()

In [3]:
## changing the width of cells
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [4]:
floatX = np.float32

## Bayesian model
God generates "forces" $Y_i\sim Norm(\mu, \lambda^2)$ for $i\in\{0, \ldots N-1\}$.
Afterwards, Nature generates observations $Z^{(0)}_i,Z^{(1)}_i \sim Norm(Y_i, \sigma^2)$. 

### Generate artificial data

In [19]:
μ_true = 1/2
λ_true = 1.
σ_true = 0.2
N = 100

Y_true = Normal(μ_true, λ_true).sample(N)
Z = Normal(loc=Y_true, scale=σ_true).sample(2)
Z = tf.transpose(Z)

### Change of variables
We have constraints $\lambda, \sigma >0$. To get rid of them we introduce
$$\beta = \log\lambda^2,\,\gamma=\log\sigma^2 \quad\text{i.e.}\quad \lambda = e^{\beta/2}, \sigma = e^{\gamma/2}.$$
We take our model paramters to be $\theta := (\mu, \beta, \gamma)$

In [5]:
def get_μλσ(θ):
    μ, β, γ = θ
    λ = exp(β/2)
    σ = exp(γ/2)
    return μ, λ, σ

### Given distributions
$\newcommand{P}{\mathbb{P}}$
$\newcommand{R}{\mathbb{R}}$
$\newcommand{Z}{\mathbb{Z}}$
$\newcommand{E}{\mathbb{E}}$
#### $\P(\theta)$ - prior on model parameters

In [22]:
μ_prior = Normal(loc=0., scale=1.)
β_prior = Normal(loc=0., scale=1.)
γ_prior = Normal(loc=0., scale=1.)

def θ_log_prior(θ) -> "scalar":
    μ, β, γ = θ
    return μ_prior.log_prob(μ) + β_prior.log_prob(β) + γ_prior.log_prob(γ)

In [24]:
## test
θ_log_prior((0., 0., 0.))

<tf.Tensor: id=255, shape=(), dtype=float32, numpy=-2.7568154>

#### $\P(Y_i\mid \theta)$ - "prior" on latent variables

In [30]:
def Y_log_priors(Y, θ):
    μ, λ, σ = get_μλσ(θ)
    return Normal(μ, λ).log_prob(Y)

In [32]:
## test
Y_log_priors(Y_true, (0.,0.,0.)).shape

TensorShape([Dimension(100)])

#### $\P(Z_i\mid Y_i,\ \theta)$ - "likelihood"

In [35]:
def Z_log_probas(Z, Y, θ):
    """
    Args:
        Z: tensor of shape `[N, 2]`
        Y: tensor of shape `[N]`
        θ: tuple of model parameters
    Returns:
        tensor of shape `[N]`
    """
    μ, λ, σ = get_μλσ(θ)
    return tf.reduce_sum(
        Normal(loc=Y[:, None], scale=σ).log_prob(Z),
        axis = -1
    )

In [37]:
## test
Z_log_probas(Z, Y_true, (0.,0.,0.)).shape

TensorShape([Dimension(100)])

### EM-algorithm

#### Variables

In [39]:
Y = tf.Variable(zeros(Z.shape[0]))

In [40]:
θ = (
    tf.Variable(0., name = "mu"),
    tf.Variable(0., name = "beta"),
    tf.Variable(0., name = "gamma"),
)

#### E-step --- MCMC

## Tensorflow Variables for Model and Data

In [9]:
def get_vector_variable(name, N_init = 10):
    X = tf.Variable(
        name = name, 
        initial_value= rnd.normal(size = N_init),
        dtype = floatX,
        validate_shape = False,       
    )
    X.set_shape([None])
    sess.run(X.initializer)
    return X

In [10]:
M = get_vector_variable("M")
X = get_vector_variable("X")
Y = get_vector_variable("Y")

In [11]:
log_sigma_2 = tf.Variable(
    np.log(0.1), name = "log_sigma_2", dtype = floatX
)
sess.run(log_sigma_2.initializer)

In [12]:
sigma_2 = tf.exp(log_sigma_2)

## Priors

### Prior on $M$

In [13]:
M_prior = SimpleNamespace()

In [14]:
M_prior.log_lhds = tf.distributions.Normal(
    loc = 0.0, scale = 1.0,
).log_prob(M)

In [15]:
M_prior.log_lhd = tf.reduce_sum(M_prior.log_lhds)

### Prior on $\sigma^2$
We have chosen the "probability density" of $\log(\sigma^2)$ to be constant, so we can ignore this.

## Likelihoods

### Observations of $X$

In [16]:
X_obs = SimpleNamespace()

In [17]:
X_obs.log_lhds = tf.distributions.Normal(
    loc = M, scale = sigma_2**(1/2),
).log_prob(X)

In [18]:
X_obs.log_lhd = tf.reduce_sum(X_obs.log_lhds)

In [20]:
sess.run(X_obs.log_lhd)

-66.665054

### Observations of $Y$

In [21]:
Y_obs = SimpleNamespace()

In [22]:
Y_obs.log_lhds = tf.distributions.Normal(
    loc = M, scale = sigma_2**(1/2),
).log_prob(Y)

In [23]:
Y_obs.log_lhd = tf.reduce_sum(Y_obs.log_lhds)

In [24]:
sess.run(Y_obs.log_lhd)

-70.45

## Posterior

In [25]:
post = SimpleNamespace()

In [26]:
post.log_lhd = sum(
    x.log_lhd for x in [M_prior, X_obs, Y_obs]
)

In [35]:
## We will need later the following
post.M_losses =  - sum(
    x.log_lhds for x in [M_prior, X_obs, Y_obs]
)

In [28]:
sess.run(post.log_lhd)

-148.53954

In [29]:
post.grad_M, post.grad_log_sigma_2 = tf.gradients(
    post.log_lhd, [M, log_sigma_2]
)

In [30]:
sess.run([post.grad_M, post.grad_log_sigma_2])

[array([-38.458435 ,  -1.6533003,  -4.608013 , -25.780334 ,  17.529793 ,
        -11.299027 ,  -1.7531624, -25.602997 , -15.927553 ,   4.90258  ],
       dtype=float32), 131.76213]

In [31]:
post

namespace(M_losses=<tf.Tensor 'Neg:0' shape=(?,) dtype=float32>, grad_M=<tf.Tensor 'gradients/AddN_2:0' shape=(?,) dtype=float32>, grad_log_sigma_2=<tf.Tensor 'gradients/Exp_grad/mul:0' shape=() dtype=float32>, log_lhd=<tf.Tensor 'add_2:0' shape=() dtype=float32>)

## Generate Data

In [32]:
N = 1000

sigma_val = 0.1
M_val = rnd.normal(size = N)

In [33]:
X_val = rnd.normal(loc = M_val, scale = sigma_val)

In [34]:
Y_val = rnd.normal(loc = M_val, scale = sigma_val)

#### Initialize variables

In [38]:
def initialize_model_variables(repetitions = 1):
    ptf.assign_to(X, list(X_val) * repetitions )
    ptf.assign_to(Y, list(Y_val) * repetitions )

    ptf.assign_to(M, np.zeros(N * repetitions))
    ptf.assign_to(log_sigma_2, 0.0)

initialize_model_variables()

## Minimize posterior

In [42]:
learning_rate = tf.placeholder(dtype = floatX, name = "learning_rate")

In [43]:
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

In [44]:
loss = - post.log_lhd

In [45]:
train_step = optimizer.minimize(
    loss = loss,
    var_list = [M, log_sigma_2]
)

In [46]:
global_variables_initializer = tf.global_variables_initializer()

In [47]:
sess.run(global_variables_initializer)
initialize_model_variables()

In [48]:
sess.run(loss)

3812.8481

In [49]:
for i in range(10000):
    sess.run(train_step, {learning_rate: 0.001})
    if i % 1000 == 0:
        print(i, sess.run(loss))

0 3811.1738
1000 2215.087
2000 452.67233
3000 -544.1853
4000 -945.8364
5000 -1007.42737
6000 -1008.5939
7000 -1008.59485
8000 -1008.595
9000 -1008.595


Here is the value of $\hat\sigma_{mle}$ we get by minimizing the posterior

In [50]:
sess.run(sigma_2)**(1/2)

0.07106736081155918

As expected, it is approximately equal to $\frac{\sigma}{\sqrt 2}$

In [51]:
sigma_val / (2 ** (1/2))

0.07071067811865475

## MCMC integration in $M$ and maximization in $\sigma$

In [52]:
mh = SimpleNamespace()

In [53]:
## size of the Metropolis-Hastings update step
mh.step_size = tf.placeholder_with_default(
    input = np.array(0.01, dtype = floatX),
    shape = [],
    name = "Metropolis_Hastings_step_size"
)

In [54]:
mh.M_suggest = tf.distributions.Normal(
    loc = M, scale = mh.step_size,
).sample()

In [56]:
def mh_update_M( step_size):
    """Update `M` using Metropolis-Hastings algorithm.
    """
    M_current = sess.run(M)
    loss_current = sess.run(post.M_losses)
    
    M_suggest = rnd.normal(M_current, scale = step_size)
    loss_suggest = sess.run(post.M_losses, {M: M_suggest})
    
    ## following Metropolis-Hastings, we decide whether we make the step or not
    step_or_not = rnd.binomial(
        n = 1,
        p = np.minimum(np.exp(loss_current - loss_suggest), 1)
    )
    
    #print(step_or_not)

    M_new = np.where(step_or_not, M_suggest, M_current)
    
    ptf.assign_to(M, M_new)
    

In [57]:
mh_update_M(0.1)

In [58]:
sigma_train_step = optimizer.minimize(
    loss = loss,
    var_list = [log_sigma_2]
)

In [59]:
global_variables_initializer = tf.global_variables_initializer()

In [63]:
sess.run(global_variables_initializer)
initialize_model_variables(repetitions = 10)

In [61]:
for i in range(1000):
    mh_update_M(0.1)

In [64]:
for n in range(10000):
    for i in range(10):
        mh_update_M(0.1)

    for i in range(10):
        sess.run(sigma_train_step, {learning_rate: 0.001})

    if n% 100 == 0:
        print(n, sess.run(loss), sess.run(sigma_2)**(1/2))

0 37575.65 0.9950579175434462
100 26016.477 0.564632083810382
200 16670.752 0.34437706496697895
300 8215.945 0.2155450472282406
400 1399.815 0.14395926031397557
500 -2116.42 0.11039035672224723
600 -2958.8125 0.10130012991655403
700 -3117.6748 0.10067872980099953
800 -3112.3457 0.10067563548118409
900 -3127.5508 0.1006624338316549
1000 -3073.25 0.10049129403716629
1100 -3148.538 0.10022729991403985
1200 -3273.6455 0.0999424125959175
1300 -2996.5986 0.10114449439830578
1400 -3340.1797 0.0998167982424038
1500 -3064.123 0.10056851986197372
1600 -3191.13 0.10034653961036731
1700 -3165.1895 0.1003561589712968
1800 -3177.4111 0.10018983643794585
1900 -3129.8242 0.10114089867189303
2000 -3112.7012 0.1009413864506549
2100 -3104.0664 0.10063409578340127
2200 -3241.2637 0.1003366826371419
2300 -3186.7002 0.10006250368990346
2400 -3135.1084 0.10060619409529625
2500 -3043.5674 0.10106798051389383
2600 -3235.3262 0.10003190090548038
2700 -3097.8887 0.10015201023280908
2800 -3132.3232 0.100436729575

In [327]:
sess.run(sigma_2)**(1/2)

0.097969468401597454